In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:

# 计算两个日期之间的工作日数,非天数.
class workDays():
    def __init__(self, start_date, end_date, days_off=None) -> object:
        """days_off:休息日,默认周六日, 以0(星期一)开始,到6(星期天)结束, 传入tupple
        没有包含法定节假日,
        """
        self.start_date = start_date
        self.end_date = end_date
        self.days_off = days_off
        if self.start_date > self.end_date:
            self.start_date, self.end_date = self.end_date, self.start_date
        if days_off is None:
            self.days_off = [5, 6]
        # 每周工作日列表
        self.days_work = [x for x in range(7) if x not in self.days_off]
        self.holidays = [
            datetime.date(2018,1,1),
            datetime.date(2017,10,1),
            datetime.date(2017, 10, 2),
            datetime.date(2017, 10, 3),
            datetime.date(2017, 10, 4),
            datetime.date(2017, 10, 5),
            datetime.date(2017, 10, 6),
            datetime.date(2017, 10, 7),
            datetime.date(2017, 10, 8)
        ]
        self.weekdayChangeWorkday = [
            datetime.date(2017, 9, 30)
        ]
    def workDays(self):
        """实现工作日的 iter, 从start_date 到 end_date , 如果在工作日内,yield 日期
        """
        # 还没排除法定节假日
        tag_date = self.start_date
        while tag_date <= self.end_date:
            if tag_date.weekday() in self.days_work :
                if tag_date not in self.holidays:
                    yield tag_date
            if tag_date.weekday() in self.days_off:
                if tag_date in self.weekdayChangeWorkday:
                    yield tag_date
            tag_date += datetime.timedelta(days=1)

    def nWorkDays(self):
        """实现非工作日的 iter, 从start_date 到 end_date , 如果在非工作日内,yield 日期
        """
        tag_date = self.start_date
        while tag_date <= self.end_date:
            if tag_date.weekday() in self.days_off:
                if tag_date not in self.weekdayChangeWorkday:
                    yield tag_date
            if tag_date.weekday() in self.days_work:
                if tag_date in self.holidays:
                    yield tag_date
            tag_date += datetime.timedelta(days=1)

    def WorkDaysCount(self):
        """工作日统计,返回数字"""
        return len(list(self.workDays()))

    def nWorkDaysCount(self):
        '''非工作日统计，返回数字'''
        return  len(list(self.nWorkDays()))


    def weeksCount(self, day_start=0):
        """统计所有跨越的周数,返回数字
        默认周从星期一开始计算
        """
        day_nextweek = self.start_date
        while True:
            if day_nextweek.weekday() == day_start:
                break
            day_nextweek += datetime.timedelta(days=1)
        # 区间在一周内
        if day_nextweek > self.end_date:
            return 1
        weeks = ((self.end_date - day_nextweek).days + 1) / 7
        weeks = int(weeks)
        if ((self.end_date - day_nextweek).days + 1) % 7:
            weeks += 1
        if self.start_date < day_nextweek:
            weeks += 1
        return weeks

In [3]:
FilePath = r'C:\Users\q\Desktop\LoanOrderList.xlsx'
data = pd.read_excel(FilePath,sheetname='Sheet1')

In [4]:
data['创建时间']=data['创建时间'].astype('datetime64')
# data['评房时间']=data['评房时间'].astype('datetime64')
data['下户完成时间']=data['下户完成时间'].astype('datetime64')
# data['初审时间']=data['初审时间'].astype('datetime64')
# data['价值认定时间']=data['价值认定时间'].astype('datetime64')
data['审批复核时间']=data['审批复核时间'].astype('datetime64')
data['渠道面签时间']=data['渠道面签时间'].astype('datetime64')
# data['公证完成时间']=data['公证完成时间'].astype('datetime64')
# data['抵押完成时间']=data['抵押完成时间'].astype('datetime64')
data['审查时间']=data['审查时间'].astype('datetime64')
data['放款时间']=data['放款时间'].astype('datetime64')

In [103]:
fk = []
cjxh=[]
xhfk=[]
fkmq=[]
mqsc=[]
scfk=[]
for i in range(len(data)):
    fk.append((data.ix[i]['放款时间']-data.ix[i]['创建时间']).total_seconds()/3600/24-
                (workDays(data.ix[i]['创建时间'].date(),data.ix[i]['放款时间'].date()).nWorkDaysCount()))
    cjxh.append((data.ix[i]['下户完成时间']-data.ix[i]['创建时间']).total_seconds()/3600/24-
                (workDays(data.ix[i]['创建时间'].date(),data.ix[i]['下户完成时间'].date()).nWorkDaysCount()))
    
    xhfk.append((data.ix[i]['审批复核时间']-data.ix[i]['下户完成时间']).total_seconds()/3600/24-
                (workDays(data.ix[i]['下户完成时间'].date(),data.ix[i]['审批复核时间'].date()).nWorkDaysCount()))
    
    fkmq.append((data.ix[i]['渠道面签时间']-data.ix[i]['审批复核时间']).total_seconds()/3600/24-
                (workDays(data.ix[i]['审批复核时间'].date(),data.ix[i]['渠道面签时间'].date()).nWorkDaysCount()))
    
    mqsc.append((data.ix[i]['审查时间']-data.ix[i]['渠道面签时间']).total_seconds()/3600/24-
                (workDays(data.ix[i]['渠道面签时间'].date(),data.ix[i]['审查时间'].date()).nWorkDaysCount()))
    
    scfk.append((data.ix[i]['放款时间']-data.ix[i]['审查时间']).total_seconds()/3600/24-
                (workDays(data.ix[i]['审查时间'].date(),data.ix[i]['放款时间'].date()).nWorkDaysCount()))

In [104]:
# 创建-下户完成	下户完成-风控通过	风控通过-面签通过	面签通过-审查通过	审查通过-
data['创建-下户完成']=cjxh
data['下户完成-风控通过']=xhfk
data['风控通过-面签通过']=fkmq
data['面签通过-审查通过']=mqsc
data['审查通过-放款']=scfk
data['放款天数']=fk

In [105]:
data.head()

,订单编号,地区,借款金额,借款期限,资金方,创建时间,评房时间,下户完成时间,初审时间,价值认定时间,...,公证完成时间,抵押完成时间,审查时间,放款时间,放款天数,创建-下户完成,下户完成-风控通过,风控通过-面签通过,面签通过-审查通过,审查通过-放款
0,N171031084524890,南京市,135,12,金融资产交易所,2017-10-31 08:45:29,2017-10-31 09:04:50,2017-10-31 13:47:45,2017-10-31 14:09:59,2017-10-31 14:49:41,...,2017-11-02 10:07:59,2017-11-02 10:07:39,2017-11-02 12:24:14,2017-11-02 16:26:53,2.320417,0.209907,0.057280,1.054572,0.830150,0.168507
1,N171103100023000,厦门市,200,12,金融资产交易所,2017-11-03 10:00:12,2017-11-03 10:42:29,2017-11-03 17:44:12,2017-11-03 18:33:01,2017-11-06 09:26:52,...,2017-11-07 12:44:48,2017-11-07 12:43:50,2017-11-07 15:16:01,2017-11-07 17:19:11,2.304850,0.322222,0.675012,0.211296,1.010787,0.085532
2,N171103133030820,苏州市,73,12,金融资产交易所,2017-11-03 13:30:23,2017-11-03 14:22:01,2017-11-06 12:03:32,2017-11-06 12:30:17,2017-11-06 15:19:13,...,2017-11-07 13:58:22,2017-11-07 13:58:47,2017-11-07 16:18:42,2017-11-07 18:04:44,2.190521,0.939688,0.197153,0.735058,0.244988,0.073634
3,N171106103109300,天津市,327,3,金融资产交易所,2017-11-06 10:31:09,2017-11-06 13:13:37,2017-11-06 18:39:45,2017-11-06 19:07:50,2017-11-07 09:45:19,...,2017-11-08 12:45:48,2017-11-08 12:45:48,2017-11-08 13:30:17,2017-11-08 16:45:59,2.260301,0.339306,0.640231,0.287685,0.857176,0.135903
4,N171106110742900,厦门市,90,6,金融资产交易所,2017-11-06 11:07:36,2017-11-06 12:03:00,2017-11-07 08:42:23,2017-11-07 09:32:24,2017-11-07 10:19:24,...,2017-11-08 12:38:21,2017-11-08 12:38:31,2017-11-08 13:56:57,2017-11-08 16:55:41,2.241725,0.899155,0.072419,0.065289,1.080741,0.124120


In [106]:
data.to_excel(r'LoanOrder_Workdays_Count.xlsx',sheet_name='sheet1')